In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
PATH = './data/ruddit.csv'
TRAIN_RATIO = 0.75
TEST_VAL_RATIO = 1

dataset = pd.read_csv(PATH)
x_train, x_test_valid, y_train, y_test_valid = train_test_split(dataset["comment_text"], dataset['offensiveness_score'] , train_size=TRAIN_RATIO, random_state=0)
x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size=TEST_VAL_RATIO, random_state=0)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)
train_raw = pd.DataFrame({'text': x_train, 'score': y_train})
test_raw = pd.DataFrame({'text': x_test, 'score': y_test})
valid_raw = pd.DataFrame({'text': x_valid, 'score': y_valid})
train_raw['score'] = train_raw['score'].astype('float32')
test_raw['score'] = test_raw['score'].astype('float32')
valid_raw['score'] = valid_raw['score'].astype('float32')
del x_train, x_test, x_valid, y_train, y_test, y_valid, x_test_valid, y_test_valid
len(train_raw), len(test_raw), len(valid_raw)

Dataset Cleaning

In [ ]:
wl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    pattern = re.compile('[^a-zA-Z]')
    text = pattern.sub(' ', text)
    text = text.lower()
    text = text.split()
    # text = [wl.lemmatize(word) for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

def clean_dataset(ds):
    ds['text'] = ds['text'].apply(preprocess_text)
    return ds

train = clean_dataset(train_raw.copy())
test = clean_dataset(test_raw.copy())
valid = clean_dataset(valid_raw.copy())

Bag of words representations

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow_train = vectorizer.fit_transform(train['text']).toarray()
bow_test = vectorizer.fit_transform(test['text']).toarray()
bow_valid = vectorizer.fit_transform(valid['text']).toarray()

pad_width = ((0, 0), (0, bow_train.shape[1] - bow_test.shape[1])) 
bow_test = np.pad(bow_test, pad_width, mode='constant', constant_values=0)
pad_width = ((0, 0), (0, bow_train.shape[1] - bow_valid.shape[1])) 
bow_valid = np.pad(bow_valid, pad_width, mode='constant', constant_values=0)


bow_train.shape, bow_test.shape, bow_valid.shape

TFIDF representations

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(train['text']).toarray()
tfidf_test = vectorizer.fit_transform(test['text']).toarray()
tfidf_valid = vectorizer.fit_transform(valid['text']).toarray()

pad_width = ((0, 0), (0, tfidf_train.shape[1] - tfidf_test.shape[1])) 
tfidf_test = np.pad(tfidf_test, pad_width, mode='constant', constant_values=0)
pad_width = ((0, 0), (0, tfidf_train.shape[1] - tfidf_valid.shape[1])) 
tfidf_valid = np.pad(tfidf_valid, pad_width, mode='constant', constant_values=0)

tfidf_train.shape, tfidf_test.shape, tfidf_valid.shape

Embeddings representations

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api 

# print(list(api.info()['models'].keys()))
w2vec_google_news_model = api.load('word2vec-google-news-300')
glove_twitter_model = api.load('glove-wiki-gigaword-300')
fasttext_wiki_news_model = api.load('fasttext-wiki-news-subwords-300')


In [ ]:
tokenized_train = [text.split() for text in train['text']]
tokenized_test = [text.split() for text in test['text']]

In [ ]:
def vectorize(sentence_tokens, w2v_model, vector_size=300):
    words_vecs = [w2v_model[word] for word in sentence_tokens if word in w2v_model]
    if len(words_vecs) == 0:
        return np.zeros(vector_size)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

train_fasttext = ([vectorize(tokens, fasttext_wiki_news_model) for tokens in tokenized_train])
test_fasttext = ([vectorize(tokens, fasttext_wiki_news_model) for tokens in tokenized_test])
train_v2w = ([vectorize(tokens, w2vec_google_news_model) for tokens in tokenized_train])
test_v2w = ([vectorize(tokens, w2vec_google_news_model) for tokens in tokenized_test])
train_glove = ([vectorize(tokens, glove_twitter_model) for tokens in tokenized_train])
test_glove = ([vectorize(tokens, glove_twitter_model) for tokens in tokenized_test])


LLMs representations

In [ ]:
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import torch

def bert_embeddings(ds):
    # Tokenize the texts
    tokenized_inputs = tokenizer(list(ds), truncation=True, padding=True, return_tensors="pt")

    # Generate BERT embeddings
    with torch.no_grad():
        model_output = model(**tokenized_inputs)

    # Extract the embeddings
    embeddings = model_output.last_hidden_state
    
    return embeddings.mean(dim=1).numpy()

In [ ]:
train_raw['text']

In [ ]:
train_bert_raw = bert_embeddings(train_raw['text'])
test_bert_raw = bert_embeddings(test_raw['text'])

In [9]:
train_bert = bert_embeddings(train['text'])
test_bert = bert_embeddings(test['text'])

KeyboardInterrupt: 

In [ ]:
import torch

def bert_text_embeddings(input_text):
    input_ids = tokenizer.encode(input_text, add_special_tokens=True)
    input_ids = torch.tensor([input_ids])

    with torch.no_grad():
        last_hidden_states = model(input_ids)[0] # Models outputs are now tuples
    last_hidden_states = last_hidden_states.mean(1)

    return last_hidden_states

In [ ]:
datasets = {
    'bow': (bow_train, bow_test, bow_valid),
    'tfidf': (tfidf_train, tfidf_test, tfidf_valid),
    'fasttext': (train_fasttext, test_fasttext),
    'word2vec': (train_v2w, test_v2w),
    'glove': (train_glove, test_glove)
}

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

linear_reg = LinearRegression()
log_reg = LogisticRegression()
svr_reg = SVR(kernel = 'rbf')
mlp_reg = MLPRegressor(random_state=1, max_iter=500)
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)


In [ ]:
model_train = train_bert
# log_reg.fit(model_train, train['score'])
# linear_reg.fit(model_train, train['score'])
svr_reg.fit(model_train, train['score'])
# mlp_reg.fit(model_train, train['score'])
# rf_reg.fit(model_train, train['score'])

In [ ]:
svr_preds = svr_reg.predict(test_bert)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
mean_squared_error(test['score'], svr_preds)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
mean_squared_error(test['score'], svr_preds)
